## Generating C code for the scalar wave equation in Cartesian coordinates

### NRPy+ Source Code for this module: [ScalarWave/ScalarWave_RHSs.py](../edit/ScalarWave/ScalarWave_RHSs.py); [ScalarWave/InitialData_PlaneWave.py](../edit/ScalarWave/InitialData_PlaneWave.py)

## Problem Statement

We wish to numerically solve the scalar wave equation in Cartesian coordinates:
$$\partial_t^2 u = c^2 \nabla^2 u \text{,}$$
where $u$ (the amplitude of the wave) is a function of time and space: $u = u(t,x,y,...)$ (spatial dimension as-yet unspecified) and $c$ is the wave speed,

subject to some initial condition
$$u(0,x,y,...) = f(x,y,...)$$

and suitable, approximate spatial boundary conditions.

Define 
$$v(t,x,y,...) = \partial_t u(t,x,y,...)$$

In this way, the second-order PDE is reduced to a set of two coupled first-order PDEs

\begin{align}
\partial_t u &= v \\
\partial_t v &= c^2 \nabla^2 u.
\end{align}

**We refer to the right-hand sides of the above two equations as the "scalar wave RHSs".** In the following sections we will 

1. Use NRPy+ to cast the RHS expressions into highly efficient C code, 
    1. first in one spatial dimension with fourth-order finite differences,
    1. and then in three spatial dimensions with tenth-order finite differences.
1. Use NRPy+ to generate monochromatic plane-wave initial data for the scalar wave equation, where the wave propagates in an arbitrary direction.

We will find that breaking the second-order time derivative into two first-order, coupled equations as above is quite convenient numerically, as it enables us to step forward in time using the Method of Lines. This method enables us to handle timestepping the same way we would handle an ODE integration, allowing us to use [Runge-Kutta methods](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods) for timestepping. 

As for the $\nabla^2 u$ term, spatial derivatives are handled in NRPy+ via [finite differencing](https://en.wikipedia.org/wiki/Finite_difference).

We will sample the solution $\{u,v\}$ at discrete, uniformly-sampled points in space and time. For simplicity, let's assume that we consider the wave equation in one spatial dimension. Then the solution at any sampled point in space and time is given by
$$u^n_i = u(t_n,x_i) = u(t_0 + n \Delta t, x_0 + i \Delta x),$$
where $\Delta t$ and $\Delta x$ represent the temporal and spatial resolution, respectively. $v^n_i$ is sampled at the same points in space and time.

### Spatial Derivatives: the $\nabla^2$ Operator

To minimize complication, we will first restrict ourselves to solving the wave equation in one spatial dimension, so
$$\nabla^2 u = \partial_x^2 u.$$
Extension of this operator to higher spatial dimensions is straightforward, particularly when using NRPy+.

As was discussed in [the finite difference section of the tutorial](Tutorial-Finite_Difference_Derivatives.ipynb), NRPy+ approximates derivatives using [finite difference methods](),  the second-order derivative $\partial_x^2$ accurate to fourth-order in uniform grid spacing $\Delta x$ (from fitting the unique 4th-degree polynomial to 5 sample points of $u$) is given by
\begin{equation}
\left[\partial_x^2 u(t,x)\right]_j = \frac{1}{(\Delta x)^2}
\left(
-\frac{1}{12} \left(u_{j+2} + u_{j-2}\right) 
+ \frac{4}{3}  \left(u_{j+1} + u_{j-1}\right)
- \frac{5}{2} u_j \right)
+ \mathcal{O}\left((\Delta x)^4\right).
\end{equation}

In [1]:
# Step P1: Import needed NRPy+ core modules:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
from outputC import *

In [2]:
# Step P2: Define the C parameter wavespeed. The `wavespeed`
#          variable is a proper SymPy variable, so it can be
#          used in below expressions. In the C code, it acts
#          just like a usual parameter, whose value is 
#          specified in the parameter file.
thismodule = "ScalarWave"
wavespeed = par.Cparameters("REAL",thismodule,"wavespeed")

# Step 1: Set the spatial dimension parameter, and then read
#         the parameter as DIM.
par.set_parval_from_str("grid::DIM",1)
DIM = par.parval_from_str("grid::DIM")

# Step 2: Set the finite differencing order to 4.
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER",4)

# Step 3: Register gridfunctions that are needed as input
#         to the scalar wave RHS expressions.
uu, vv = gri.register_gridfunctions("EVOL",["uu","vv"])

# Step 4: Declare the rank-2 indexed expression \partial_{ij} u,
#         which is symmetric about interchange of indices i and j
#         Derivative variables like these must have an underscore
#         in them, so the finite difference module can parse the
#         variable name properly.
uu_dDD = ixp.declarerank2("uu_dDD","sym12")

# Step 5: Define right-hand sides for the evolution.
uu_rhs = vv
vv_rhs = 0
for i in range(DIM):
    vv_rhs += wavespeed*wavespeed*uu_dDD[i][i]

vv_rhs = sp.simplify(vv_rhs)

# Step 6: Generate C code for scalarwave evolution equations,
#         print output to the screen (standard out, or stdout).
fin.FD_outputC("stdout",
               [lhrh(lhs=gri.gfaccess("rhs_gfs","uu"),rhs=uu_rhs),
                lhrh(lhs=gri.gfaccess("rhs_gfs","vv"),rhs=vv_rhs)])

/*
 *  Original SymPy expressions:
 *  "[const double uu_dDD00 = invdx0**2*(-5*uu/2 + 4*uu_i0m1/3 - uu_i0m2/12 + 4*uu_i0p1/3 - uu_i0p2/12),
 *    rhs_gfs[IDX2(UUGF, i0)] = vv,
 *    rhs_gfs[IDX2(VVGF, i0)] = uu_dDD00*wavespeed**2]"
 */
{
   const double uu_i0m2 = in_gfs[IDX2(UUGF, i0-2)];
   const double uu_i0m1 = in_gfs[IDX2(UUGF, i0-1)];
   const double uu = in_gfs[IDX2(UUGF, i0)];
   const double uu_i0p1 = in_gfs[IDX2(UUGF, i0+1)];
   const double uu_i0p2 = in_gfs[IDX2(UUGF, i0+2)];
   const double vv = in_gfs[IDX2(VVGF, i0)];
   const double uu_dDD00 = pow(invdx0, 2)*(-(5.0 / 2.0)*uu + ((4.0 / 3.0))*uu_i0m1 - (1.0 / 12.0)*uu_i0m2 + ((4.0 / 3.0))*uu_i0p1 - (1.0 / 12.0)*uu_i0p2);
   rhs_gfs[IDX2(UUGF, i0)] = vv;
   rhs_gfs[IDX2(VVGF, i0)] = uu_dDD00*pow(wavespeed, 2);
}


**Success!** Notice that indeed NRPy+ was able to compute the spatial derivative operator,
\begin{equation}
\left[\partial_x^2 u(t,x)\right]_j \approx \frac{1}{(\Delta x)^2}
\left(
-\frac{1}{12} \left(u_{j+2} + u_{j-2}\right) 
+ \frac{4}{3}  \left(u_{j+1} + u_{j-1}\right)
- \frac{5}{2} u_j \right),
\end{equation}
correctly (easier to read in the "Original SymPy expressions" comment block at the top of the C output. Note that $\texttt{invdx0}=1/\Delta x_0$, where $\Delta x_0$ is the (uniform) grid spacing in the zeroth, or $x_0$ direction.

Now let's repeat the process, except at **10th** finite difference order, for the **3-spatial-dimension** scalar wave equation, with SIMD enabled:

In [3]:
# Step 1: Define the C parameter wavespeed. The `wavespeed`
#         variable is a proper SymPy variable, so it can be
#         used in below expressions. In the C code, it acts
#         just like a usual parameter, whose value is 
#         specified in the parameter file.
wavespeed = par.Cparameters("REAL",thismodule,"wavespeed")

# Step 2: Enable SIMD
par.set_parval_from_str("outputC::SIMD_enable",True)

# Step 3: Set the spatial dimension parameter 
#         to *FOUR* this time, and then read
#         the parameter as DIM.
par.set_parval_from_str("grid::DIM",3)
DIM = par.parval_from_str("grid::DIM")

# Step 4: Set the finite differencing order to 10.
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER",10)

# Step 5a: Reset gridfunctions registered in 1D case above,
#          to avoid NRPy+ throwing an error about double-
#          registering gridfunctions, which is not allowed.
gri.glb_gridfcs_list = []

# Step 5b: Register gridfunctions that are needed as input
#          to the scalar wave RHS expressions.
uu, vv = gri.register_gridfunctions("EVOL",["uu","vv"])

# Step 6: Declare the rank-2 indexed expression \partial_{ij} u,
#         which is symmetric about interchange of indices i and j
#         Derivative variables like these must have an underscore
#         in them, so the finite difference module can parse the
#         variable name properly.
uu_dDD = ixp.declarerank2("uu_dDD","sym12")

# Step 7: Define right-hand sides for the evolution.
uu_rhs = vv
vv_rhs = 0
for i in range(DIM):
    vv_rhs += wavespeed*wavespeed*uu_dDD[i][i]

# Step 8: Simplify the expression for c^2 \nabla^2 u (a.k.a., vv_rhs):
vv_rhs = sp.simplify(vv_rhs)

# Step 9: Generate C code for scalarwave evolution equations,
#         print output to the screen (standard out, or stdout).
fin.FD_outputC("stdout",
               [lhrh(lhs=gri.gfaccess("rhs_gfs","uu"),rhs=uu_rhs),
                lhrh(lhs=gri.gfaccess("rhs_gfs","vv"),rhs=vv_rhs)])

initialize_param() minor warning: Did nothing; already initialized parameter ScalarWave::wavespeed
/*
 *  Original SymPy expressions:
 *  "[const REAL_SIMD_ARRAY uu_dDD11 = invdx1**2*(-5269*uu/1800 + 5*uu_i0_i1m1_i2/3 - 5*uu_i0_i1m2_i2/21 + 5*uu_i0_i1m3_i2/126 - 5*uu_i0_i1m4_i2/1008 + uu_i0_i1m5_i2/3150 + 5*uu_i0_i1p1_i2/3 - 5*uu_i0_i1p2_i2/21 + 5*uu_i0_i1p3_i2/126 - 5*uu_i0_i1p4_i2/1008 + uu_i0_i1p5_i2/3150),
 *    const REAL_SIMD_ARRAY uu_dDD00 = invdx0**2*(-5269*uu/1800 + 5*uu_i0m1_i1_i2/3 - 5*uu_i0m2_i1_i2/21 + 5*uu_i0m3_i1_i2/126 - 5*uu_i0m4_i1_i2/1008 + uu_i0m5_i1_i2/3150 + 5*uu_i0p1_i1_i2/3 - 5*uu_i0p2_i1_i2/21 + 5*uu_i0p3_i1_i2/126 - 5*uu_i0p4_i1_i2/1008 + uu_i0p5_i1_i2/3150),
 *    const REAL_SIMD_ARRAY uu_dDD22 = invdx2**2*(-5269*uu/1800 + 5*uu_i0_i1_i2m1/3 - 5*uu_i0_i1_i2m2/21 + 5*uu_i0_i1_i2m3/126 - 5*uu_i0_i1_i2m4/1008 + uu_i0_i1_i2m5/3150 + 5*uu_i0_i1_i2p1/3 - 5*uu_i0_i1_i2p2/21 + 5*uu_i0_i1_i2p3/126 - 5*uu_i0_i1_i2p4/1008 + uu_i0_i1_i2p5/3150),
 *    const REAL_SIMD_ARRA

### NRPy+ Module Code Validation

Here, as a code validation check, we verify agreement in the SymPy expressions for the RHSs of the Scalar Wave equation (i.e., uu_rhs and vv_rhs) between
1. this tutorial and 
2. the NRPy+ ScalarWave/ScalarWave_RHSs.py module.

In [4]:
# Step 10: We already have SymPy expressions for uu_rhs and vv_rhs in
#          terms of other SymPy variables. Even if we reset the list
#          of NRPy+ gridfunctions, these *SymPy* expressions for
#          uu_rhs and vv_rhs *will remain unaffected*. 
# 
#          Here, we will use the above-defined uu_rhs and vv_rhs to 
#          validate against the same expressions in the 
#          ScalarWave/ScalarWave_RHSs.py module,
#          to ensure consistency between this tutorial 
#          (historically speaking, the tutorial was written first) 
#          and the ScalarWave_RHSs.py module itself.
#
# Reset the list of gridfunctions, as registering a gridfunction
#   twice will spawn an error.
gri.glb_gridfcs_list = []

# Step 11: Call the ScalarWave_RHSs() function from within the
#         ScalarWave/ScalarWave_RHSs.py module,
#         which should do exactly the same as in Steps 1-10 above.
import ScalarWave.ScalarWave_RHSs as swrhs
swrhs.ScalarWave_RHSs()

# Step 12: Consistency check between the tutorial module above
#         and the ScalarWave_RHSs() function from within the
#         ScalarWave/ScalarWave_RHSs.py module.
# It is SAFE to ignore the warning from re-initializing the parameter RMAX.
print("^^^ Ignore the minor warning above. ^^^\n")
print("Consistency check between ScalarWave tutorial and NRPy+ module:")
print("uu_rhs - swrhs.uu_rhs: Should be zero: ",sp.simplify(uu_rhs - swrhs.uu_rhs))
print("vv_rhs - swrhs.vv_rhs: Should be zero: ",sp.simplify(vv_rhs - swrhs.vv_rhs))

^^^ Ignore the minor warning above. ^^^

Consistency check between ScalarWave tutorial and NRPy+ module:
uu_rhs - swrhs.uu_rhs: Should be zero:  0
vv_rhs - swrhs.vv_rhs: Should be zero:  0


## Initial Data for the Scalar Wave Equation

The plane-wave solution to the scalar wave equation for a wave traveling in the $\hat{k}$ direction is
$$u(\vec{x},t) = f(\hat{k}\cdot\vec{x} - c t),$$
where $\hat{k}$ is a unit vector.

In [5]:
# Step 1: Set parameters defined in other modules
xx = gri.xx

# Step 2: Declare free parameters intrinsic to these initial data
time = par.Cparameters("REAL",thismodule,"time")
kk   = par.Cparameters("REAL",thismodule,["kk0","kk1","kk2"])

# Step 3: Normalize the k vector
kk_norm = sp.sqrt(kk[0]**2 + kk[1]**2 + kk[2]**2)

# Step 4: Compute k.x
dot_product = sp.sympify(0)
for i in range(DIM):
    dot_product += xx[i]*kk[i]
dot_product /= kk_norm

# Step 5: Set initial data for uu and vv, where vv_ID = \partial_t uu_ID.
uu_ID = sp.sin(dot_product - wavespeed*time)
vv_ID = sp.diff(uu_ID, time)

Next we verify that $f(\hat{k}\cdot\vec{x} - c t)$ satisfies the wave equation, by computing
$$\left(c^2 \nabla^2 - \partial_t^2 \right)\ f\left(\hat{k}\cdot\vec{x} - c t\right),$$
and confirming the result is exactly zero.

In [6]:
sp.simplify(wavespeed**2*(sp.diff(uu_ID,xx[0],2) + 
                          sp.diff(uu_ID,xx[1],2) + 
                          sp.diff(uu_ID,xx[2],2))
            - sp.diff(uu_ID,time,2))

0

### NRPy+ Module Code Validation

As a code validation check, we will verify agreement in the SymPy expressions for plane-wave initial data for the Scalar Wave equation between
1. this tutorial and 
2. the NRPy+ ScalarWave/InitialData_PlaneWave.py module.

In [7]:
# We just defined SymPy expressions for uu_ID and vv_ID in
# terms of other SymPy variables. Here, we will use the 
# above-defined uu_ID and vv_ID to validate against the 
# same expressions in the ScalarWave/InitialData_PlaneWave.py 
# module, to ensure consistency between this tutorial 
# (historically speaking, the tutorial was written first) 
# and the PlaneWave ID module itself.
# 
# Step 6: Call the InitialData_PlaneWave() function from within the
#         ScalarWave/InitialData_PlaneWave.py module,
#         which should do exactly the same as in Steps 1-5 above.
import ScalarWave.InitialData_PlaneWave as swid
swid.InitialData_PlaneWave()

# Step 7: Consistency check between the tutorial module above
#         and the InitialData_PlaneWave() function from within the
#         ScalarWave/InitialData_PlaneWave.py module.
# It is SAFE to ignore the warning from re-initializing the parameter RMAX.
print("Consistency check between ScalarWave tutorial and NRPy+ module:")
print("uu_ID - swid.uu_ID: Should be zero: ",sp.simplify(uu_ID - swid.uu_ID))
print("vv_ID - swid.vv_ID: Should be zero: ",sp.simplify(vv_ID - swid.vv_ID))

Consistency check between ScalarWave tutorial and NRPy+ module:
uu_ID - swid.uu_ID: Should be zero:  0
vv_ID - swid.vv_ID: Should be zero:  0
